In [1]:
#import libraries
import selenium 
from selenium import webdriver
import time
import pickle
import itertools
from bs4 import BeautifulSoup

#setup and return driver
def setup_driver():
    #add header(User-Agent) so website thinks it's an authentic request
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("--user-agent=Chrome/74.0.3729.131")
    driver = webdriver.Chrome("./chromedriver", options=chrome_options)
    
    driver.get("https://investovc.com/fundadores")
    
    return driver

#get all of the fund elements
def get_funds(driver):
    funds_container = driver.find_elements_by_xpath("//div[@class='panel-group']")[0]
    funds = funds_container.find_elements_by_xpath("//div[contains(@class, 'panel-heading panel-head')]")
    return funds

#get the fund title
def get_fund_title(ind, fund):
    title_elm = fund.find_elements_by_xpath("//h3[@class='panel-title']")[ind]
    return title_elm.get_attribute("innerHTML")

def get_startups_for_fund(ind, driver):
    desc_elm = driver.find_elements_by_xpath("//h4[@class='panel-title']")[ind]
    desc_elm = desc_elm.get_attribute("innerHTML")
    desc_elm = [int(s) for s in desc_elm.split() if s.isdigit()][0]
    return desc_elm

#clean the description
def clean_description(description):
    description= description.replace(u'\xa0', u' ')
    description = description.replace('Show more>', '')
    description =description.replace('Show less>', '')
    description = description.replace('...', '')
    return description 

def clean_text(loc):
    loc = loc.replace('\n',"")
    loc = loc.replace('<p>',"")
    loc = loc.replace('</p>',"")
    loc = loc.replace('-',"")
    return loc
   
#open original page, let it load
c_driver = setup_driver()
time.sleep(5)

all_info = []
funds = get_funds(c_driver)
funds_len = len(funds)

#go through all of the funds in the current page, and open them
for ind in range(funds_len):
    
    #click to open
    c_driver.execute_script("arguments[0].scrollIntoView();", funds[ind])
    fund_title = get_fund_title(ind, funds[ind])
    fund_amt = get_startups_for_fund(ind, c_driver)
    funds[ind].click()

    all_info.append({"fund_name": fund_title, "fund_amt":fund_amt})

#description
time.sleep(10)
elms = c_driver.find_elements_by_xpath("//p[@class='more smsl']")
for i, elm in enumerate(elms):
    text = elm.get_attribute("innerHTML")
    all_info[i]["description"] = BeautifulSoup(clean_description(text)).get_text()

#location
count = 0
count_url = 0
in_between = 0
elms = c_driver.find_elements_by_xpath("//div[@class='tcb-info-block']")
for i, elm in enumerate(elms):
    text = elm.get_attribute("innerHTML")
    in_between +=1
    if "Location" in text:
        if in_between > 23:
            all_info[count]["location"] = "Not Available on Site"
            count +=1
        all_info[count]["location"] = clean_text(elms[i+1].get_attribute("innerHTML"))
        count +=1
        in_between = 0
    if "Web Presence" in text:
        par = elm.find_element_by_xpath('..')
        html = par.get_attribute("innerHTML")
        soup = BeautifulSoup(html)
        all_info[count_url]["url"] = soup.a.get("href")
        count_url += 1

In [2]:
all_info

[{'fund_name': 'Y Combinator',
  'fund_amt': 1686,
  'description': "Y Combinator is a startup accelerator based in Mountain View, CA.\n\nIn 2005, Y Combinator developed a new model of startup funding. Twice a year they invest a small amount of money ($120K) in a large number of startups (most recently 68). The startups move to Silicon Valley for 3 months. The YC partners work closely with each company to get them into the best possible shape and refine their pitch to investors. Each cycle culminates in Demo Day, when the startups present their business plans to a \xa0carefully selected, audience of investors.\n\nSuccessful exits by Y Combinator funded companies include \n[Reddit](http://www.crunchbase.com/company/reddit), [Heroku](http://www.crunchbase.com/company/heroku) and [OMGPOP](http://www.crunchbase.com/company/omgpop).\n\nOther successful companies that went through Y Combinator include [Dropbox](http://www.crunchbase.com/company/dropbox), [Airbnb](http://www.crunchbase.com/co

In [99]:
#click on all the more buttons
time.sleep(3)
more_buttons = c_driver.find_elements_by_xpath("//div[@class='investor_block investor-more-investments']")
for more_button in more_buttons:
    c_driver.execute_script("arguments[0].scrollIntoView();", more_button)
    more_button.click()
time.sleep(15)

In [100]:
#get all of the 
startups = c_driver.find_elements_by_xpath("//div[@class='investor_block  quickinfo quickinfo-startup ']")
#to be continued
for info in all_info:
    

15291
